In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import csv
from numba import jit, prange
import networkx as nx
from networkx.algorithms import bipartite
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
from time import time
import pickle
from numpy import ma

In [2]:
input_file = 'pizzas/b_little_bit_of_everything.in'
!ls pizzas

b_little_bit_of_everything.in  d_many_pizzas.out
b_little_bit_of_everything.out d_many_pizzas.pk
b_little_bit_of_everything.pk  data_Structures.py
c_many_ingredients.in          e_many_teams.in
c_many_ingredients.out         e_many_teams.out
c_many_ingredients.pk          e_many_teams.pk
d_many_pizzas.in


# Get M matrix

In [3]:
def read(input_file):
    pizza_dict = {}
    with open(input_file, newline='') as fin:
        reader = csv.reader(fin, delimiter=' ')
        n_team = {}
        _, n_team[2], n_team[3], n_team[4] = next(reader)
        n_team = {k:int(v) for k,v in n_team.items()}
        for pizza_id, line in enumerate(reader):
            pizza_dict[pizza_id] = line[1:]
            
    return n_team, pizza_dict

@jit(nopython=True, parallel=True)
def get_unique_ingredients_matrix(M):
    for i in prange(M.shape[0]):
        for j in range(M.shape[1]):
            M[i,j] = M[i,i] + M[j,j] - M[i,j]
        
    return M

In [4]:
for input_file in glob('pizzas/*.in'):
    print(input_file)
    t = time()
    
    n_team, pizza_dict = read(input_file)
    n_pizzas = len(pizza_dict)
    G = nx.from_dict_of_lists(pizza_dict)
    pizzas, ingredients = bipartite.sets(G)
    adj_mat = bipartite.biadjacency_matrix(G, np.arange(n_pizzas), dtype=np.int32).toarray()
    M = adj_mat.dot(adj_mat.T)
    # M = get_unique_ingredients_matrix(M)
    # By uncommenting, M can be modified here to account for the number of unique ingredients for each couple. This allows to really maximize the objective function.
    
    print('Time for M: ', (time() - t)/60, ' m')
    t = time()
    
    # Cannot write M on file because huge (something like 40GB)

pizzas/d_many_pizzas.in
Time for M:  11.15917401711146  m
pizzas/c_many_ingredients.in
Time for M:  9.992789947986603  m
pizzas/b_little_bit_of_everything.in
Time for M:  0.006496016184488932  m
pizzas/e_many_teams.in
Time for M:  10.898229630788167  m


# Full algorithm
Add functions implementing the algorithm (still very slow and not efficient)

In [4]:
def find_best_couples(M_in):
    best_couples = []
    
    M = ma.masked_array(M_in)
    M[np.diag_indices(M.shape[0])] = ma.masked
    M[np.tril_indices(M.shape[0])] = ma.masked

    for _ in range(int(np.ceil(M.shape[0]/2))):
        i, j = np.unravel_index(M.argmax(), M.shape)
        M[(i,j), :] = ma.masked
        M[:, (i,j)] = ma.masked

        best_couples.append((i,j))
        
    return best_couples

def assign_even_team_pizzas(remaining_couples, team_size):
    team_pizzas = []
    for i in range(n_team[team_size]):
        if len(remaining_couples) >= team_size//2:
            delivery_list = [team_size]
            for n in range(team_size//2):
                delivery_list += [*remaining_couples.pop(0)]
                
            team_pizzas.append(delivery_list)
            
    return team_pizzas

def assign_odd_team_pizzas(remaining_couples, team_size):
    team_pizzas = []
    spare_pizza = None
    for i in range(n_team[team_size]):
        if len(remaining_couples) >= team_size//2+1:
            delivery_list = [team_size]
            for n in range(team_size//2):
                delivery_list += [*remaining_couples.pop(0)]
                
            if spare_pizza is not None:
                delivery_list += [spare_pizza]
                spare_pizza = None
            else:
                a, b = remaining_couples.pop(0)
                delivery_list += [a]
                spare_pizza = b
                
            team_pizzas.append(delivery_list)
            
    return team_pizzas
    

def assign_all_pizzas(best_df):
    all_pizzas = []
    all_pizzas.extend(assign_even_team_pizzas(best_couples, team_size=2))
    all_pizzas.extend(assign_even_team_pizzas(best_couples, team_size=4))
    all_pizzas.extend(assign_odd_team_pizzas(best_couples, team_size=3))
    
    all_pizzas.insert(0, [len(all_pizzas)])
    
    return all_pizzas

In [ ]:
for input_file in glob('pizzas/*.in'):
    print(input_file)
    t = time()
    
    n_team, pizza_dict = read(input_file)
    n_pizzas = len(pizza_dict)
    G = nx.from_dict_of_lists(pizza_dict)
    pizzas, ingredients = bipartite.sets(G)
    adj_mat = bipartite.biadjacency_matrix(G, np.arange(n_pizzas), dtype=np.int32).toarray()
    M = adj_mat.dot(adj_mat.T)
    M = get_unique_ingredients_matrix(M)
    # M is modified here to account for the number of unique ingredients for each couple. This allows to really maximize the objective function.
    
    print('Time for M: ', (time() - t)/60, ' m')
    t = time()
    
    best_couples = find_best_couples(M)
    all_pizzas = assign_all_pizzas(best_couples)
    
    print('Time for algo: ', (time() - t)/60, ' m')

    # Write output file
    with open(input_file[:-2]+'out', 'w') as fout:
        writer = csv.writer(fout, delimiter=' ')
        writer.writerows(all_pizzas)
    
#    with open(input_file[:-2]+'M.pickle', 'wb') as fout:
#        pickle.dump(M, fout) 
        
#    with open(input_file[:-2]+'M_unique.pickle', 'wb') as fout:
#        pickle.dump(M, fout)

pizzas/d_many_pizzas.in
Time for M:  0.09273718198140463  m


In [ ]:
# Check uniqueness of delivered pizzas
flattened = []
for line in all_pizzas[1:]:
    flattened += line[1:] 

pd.DataFrame(flattened).duplicated().values.nonzero()

In [ ]:
# Check how many pizzas to each team size
flattened = []
for line in all_pizzas[1:]:
    flattened += [line[0]] 

pd.DataFrame(flattened).value_counts()

# Backup

In [ ]:

unique_ingredients = sorted(set().union(*pizza_dict.values()))

n_unique_ingredients = len(unique_ingredients)
#unique_ingredients_df = pd.DataFrame(unique_ingredients, columns=['ingredient'])

In [ ]:
M_bak = M.copy()

In [ ]:
M = M_bak.copy()

In [ ]:
best_df = pd.DataFrame(M).stack()
best_df = best_df.reset_index()
best_df.columns = ['x', 'y', 'n']
best_df = best_df[best_df['x'] < best_df['y']] # Take upper triangular part

best_df = best_df.sort_values(by='n', ascending=False).reset_index()
best_couples = best_df.loc[:, ['x', 'y']].values.tolist()

In [ ]:
best_df = pd.DataFrame(M).stack()
best_df = best_df.reset_index()
best_df.columns = ['x', 'y', 'n']
best_df = best_df[best_df['x'] < best_df['y']] # Take upper triangular part
best_df = best_df.sort_values(by='n', ascending=False).reset_index(drop=True)
best_df = best_df.groupby('x', group_keys=False).apply(lambda x: x.loc[x['n'].idxmax()])
nest_df = best_df.reset_index(drop=True).loc[:, ['x','y']]
best_couples = nest_df.values.tolist()

In [ ]:
best_couples = select_best_couples(M)

In [ ]:
all_pizzas = assign_all_pizzas(best_df)

# Write output file
with open(input_file[:-2]+'out', 'w') as fout:
    writer = csv.writer(fout, delimiter=' ')
    writer.writerows(all_pizzas)

In [ ]:
!cat pizzas/b_little_bit_of_everything.out

In [ ]:
best_couples

In [ ]:
import pandas as pd
import numba


data = read(input_file)

def get_bool(data):
    unique = list(set().union(*data))
    unique_hash = {el : i for i,el in enumerate(unique)}

    m = np.zeros((len(data), len(unique)), dtype=bool)
    for i, line in enumerate(data):
        for el in line:
            m[i, unique_hash[el]] = 1
            
    return m

def get_bool_jit(data):
    unique = list(set().union(*data))
    unique_hash = {el : i for i,el in enumerate(unique)}
    
    
    @jit(nopython=True, parallel=True)
    def fun(data, unique):
        m = np.zeros((len(data), len(unique)), dtype=bool)
        numba.
        for i in prange(len(data)):
            for el in data[i]:
                m[i, 1] = True
                
        return m
        
    return fun(data, unique)